# Arytmetyka w ciałach Galois

## Co to jest ciało Galois?

**Ciałem Galois** nazywamy ciało $(G,+,\cdot)$ o skończonej liczbie elementów. Najprostszym przykładem takiego ciała są $\mathbb{Z}_p$, gdzie $p$ jest liczbą pierwszą. *Rzędem* ciała skończonego nazywamy liczbę jego elementów. Ciała skończone tego samego rzędu są izomorficzne, tzn. istnieje pomiędzy nimi bijekcja zachowująca działania.

Kolejnym ważnym przykładem ciała Galois jest pierścień ilorazowy $\mathbb{Z}_p[X]$ / $W(X)$, gdzie $p$ jest liczbą pierwszą, a $W$ jest nierozkładalnym wielomianem monicznym stopnia $n$ (czyli takim, którego współczynnik przy $X^n$ jest równy 1).

## Pierścień $\mathbb{Z}_n$

W ciele $\mathbb{Z}$ wprowadzamy relację równoważności$\mod n$ (gdzie $n$ jest ustaloną, dodatnią liczbą naturalną):
$$a\equiv_n b\Leftrightarrow [a]_n=[b]_n$$
gdzie $[\cdot]_n$ oznacza resztę z dzielenia przez $n$.

Arytmetyka$\mod n$:$$a+b=[a+b]_n$$ $$ab=[ab]_n$$

**Pierwiastkiem pierwotnym**$\mod n$ nazywamy liczbę, której potęgi$\mod n$ dają wszystkie reszty z dzielenia przez $n$, które są względnie pierwsze z $n$. Pierwiastek pierwotny istnieje tylko dla następujących $n$:
- $n=p^k$, gdzie $p$ jest liczbą pierwszą *różną* od 2,
- $n=2p^k$, gdzie $p$ - j. w.,
- $n=2$ lub $n=4$.

Przykładowo, wszystkie reszty z dzielenia przez 5 względnie pierwsze z 5 to 4, 3, 2, 1 (zero odpada - nie jest względnie pierwsze). Kolejne potęgi $2^k\mod 5$ to:
- $2^1\mod 5=2$,
- $2^2\mod 5=4$,
- $2^3\mod 5=3$,
- $2^4\mod 5=1$.

Czyli 2 jest pierwiastkiem pierwotnym$\mod 5$.

### Sage math
Konstruujemy pierścień `R = Integers(n)` lub `R = IntegerModRing(n)`, gdzie za `n` podajemy ustaloną liczbę naturalną. Jeżeli chcemy poznać postać liczby `x` w tym pierścieniu, to piszemy `R(x)`. Inną opcją jest funkcja `mod(x, n)`.

In [1]:
from sage.all import *

R = Integers(3)
x = R(5)

print(x)
print(type(x))

2
<class 'sage.rings.finite_rings.integer_mod.IntegerMod_int'>


In [2]:
4 * x

2

In [3]:
x + 2

1

In [4]:
x ** 10

1

In [5]:
R(2 + 7)

0

In [6]:
R(2 * 4)

2

In [7]:
RR = IntegerModRing(5)
x = RR(10)

print(x)
print(type(x))

0
<class 'sage.rings.finite_rings.integer_mod.IntegerMod_int'>


In [8]:
x = mod(10, 4)

print(x)
print(type(x))

2
<class 'sage.rings.finite_rings.integer_mod.IntegerMod_int'>


### Dzielenie w arytmetyce modularnej

W momencie, gdy $p$ jest liczbą pierwszą, pierścień $\mathbb{Z}_p$ jest czymś więcej - jest ciałem, czyli każdy niezerowy element posiada *element odwrotny*, a zatem możemy zdefiniować operację dzielenia:
$$
a/b=a*b^{-1},
$$
gdzie $b^{-1}$ oznacza właśnie element odwrotny do $b$, czyli taki, że $b^{-1}*b=b*b^{-1}=1$.

W przypadku, gdy podstawą arytmetyki modularnej nie jest liczba pierwsza, to nie mamy do czynienia z ciałem - nie wszystkie elementy będą odwracalne (tzn. nie każde dzielenie jest wykonalne).

In [9]:
x = mod(4, 5) # podstawa 5 - liczba pierwsza
x / 3         # liczbą odwrotną do 3 jest 2, bo 3 * 2 mod 5 = 1, czyli 4 / 3 mod 5 = 4 * 2 mod 5 = 3

3

In [10]:
y = mod(3, 4) # podstawa 4 - nie jest liczbą pierwszą
y / 2         # każda wielokrotność 2 jest liczbą parzystą, zatem nigdy jej reszta z dzielenia przez 4 nie da 1
              # zatem 2 nie jest odwracalne

ZeroDivisionError: inverse of Mod(2, 4) does not exist

Pierwiastki pierwotne w Sage znajdujemy funkcją `primitive_root(n)`.

In [11]:
x = primitive_root(1907)

print(x)
print(type(x))

2
<class 'sage.rings.integer.Integer'>


In [12]:
primitive_root(15) # nie istnieją pierwiastki pierwotne mod 15

ValueError: no primitive root

### Python

W Pythonie nie poszalejemy - operator `%` zwraca resztę z dzielenia. I to tyle. Funkcje do arytmetyki mod $n$ można znaleźć w module **SymPy**.

In [13]:
x = 5 % 2

print(x)
print(type(x))

1
<class 'int'>


## Zadanie 1

Zaimplementować w Pythonie klasę `Zn(N)`, czyli pierścień reszt z dzielenia przez `N`. Przeładować operatory `+`, `-`, `*` i `**` tak, aby na obiektach klasy wykonywały działania mod `N`, działania dodawania i mnożenia przez `int` oraz metodę `__repr__`.

In [14]:
# your code here
class Zn:
    def __init__(self, val, N):
        self.val = val % N
        self.N = N

    def check_argument(self, argument):
        if not isinstance(argument, Zn):
            if isinstance(argument, int):
                return Zn(argument, self.N)
            raise TypeError(f"Argument operacji powinien być obiektem klasy Zn lub liczbą całkowitą! Otrzymano typ: {type(argument)}")
        elif argument.N != self.N:
            raise ValueError(f"Różne podstawy modularne N w argumentach: {self.N} oraz {argument.N}")

        return argument

    def is_negative_power(self, argument):
        return isinstance(argument, int) and argument < 0

    def find_modular_inverse(self):
        for i in range(1, self.N):
            if i * self.val % self.N == 1:
                return i

        raise ZeroDivisionError(f"{self.val} nie ma odwrotności modularnej dla podstawy modularnej {self.N}")

    def get_value(self):
        return self.val
    
    def __add__(self, other):
        other = self.check_argument(other)
        return Zn(self.val + other.val, self.N)

    def __radd__(self, other):
        return self + other

    def __sub__(self, other):
        other = self.check_argument(other)
        return Zn(self.val - other.val, self.N)

    def __rsub__(self, other):
        other = self.check_argument(other)
        return other - self

    def __mul__(self, other):
        other = self.check_argument(other)
        return Zn(self.val * other.val, self.N) 

    def __rmul__(self, other):
        return self * other

    def __pow__(self, other):
        if self.is_negative_power(other):
            power = -other
            mod_inv = self.find_modular_inverse()
            return Zn(mod_inv ** power, self.N)
        else:
            other = self.check_argument(other)
            return Zn(self.val ** other.val, self.N)

    def __rpow__(self, other):
        other = self.check_argument(other)
        return other ** self

    def __str__(self):
        return self.__repr__()
    
    def __repr__(self):
        return str(self.val)

In [15]:
# TESTY
x = Zn(2, 7)
y = Zn(10, 7)
z = Zn(14, 7)

print(x, y, z)
# 2 3 0

print(x + z, x * y, x ** y, 6 + x, x + 6, 4 * y, y * 4)
# 2 6 1 1 1 5 5

2 3 0
2 6 1 1 1 5 5


In [16]:
# odwrotności modularne
print(x ** -1)
print(y ** -1)
print(z ** -1) # błąd - 0 nie ma odwrotności modularnej!

4
5


ZeroDivisionError: 0 nie ma odwrotności modularnej dla podstawy modularnej 7

In [17]:
# ujemne potęgi
print(x ** -3)
print(y ** -5)
print(x ** -4 * y ** -7)

1
3
6


## Pierścienie ilorazowe wielomianów

Aby utworzyć pierścień ilorazowy $\mathbb{Z}_n[X]$ / $W(X)$ w Sage musimy najpierw utworzyć $\mathbb{Z}_n[X]$, czyli pierścień wielomianów o współczynnikach z $\mathbb{Z}_n$:

`R = PolynomialRing(Integers(n), 'X')`

Jeżeli w dalszej części kodu mamy zamiar korzystać z wielomianów z tego pierścienia, to dobrze jest rozdzielić nazewnictwo zmiennych niezależnych:

`X = R.gen()`

Teraz każdy wielomian zmiennej `X` będzie przez Sage traktowany jako element pierścienia `R`.

In [18]:
R = PolynomialRing(Integers(5), 'X')
X = R.gen()

X ** 6 - 13 * X ** 4 + 12 * X ** 2 - 10 * X + 6

X^6 + 2*X^4 + 2*X^2 + 1

Pierścień ilorazowy tworzymy metodą `R.quotient(W, 'x')`, gdzie `W` jest dowolnym wielomianem. Podobnie jak poprzednio, dobrze jest od razu zdefiniować `x` jako zmienną niezależną wielomianów z nowego pierścienia.

In [19]:
Rq = R.quotient(X ** 4 + 1, 'x')
x = Rq.gen()

x ** 6 - 13 * x ** 4 + 12 * x ** 2 - 10 * x + 6

x^2 + 4

In [20]:
w1 = 7 * x ** 6 + 14
w2 = 24 * x ** 4 - 5 * x ** 2 - 7 * x + 13

expand(w1 * w2)

4*x^3 + 2*x^2 + 2*x + 1

## Zadanie 2

Mając klasę `Zn(N)`, zaimplementować w Pythonie klasę `ZnW(N, W)`, czyli pierścień ilorazowy $\mathbb{Z}_n[X]$ / $W(X)$ z działaniami dodawania i mnożenia wielomianów.

Dane testowe:

\begin{align}
    w1 &= 7x^6+14x^3 \\
    w2 &= 24x^4-5x^2-7x+13 \\
    w3 &= 23x^5-3x^4+x^3+35x^2+4
\end{align}

Reprezentacja w $\mathbb{Z}_{17}[X]$ / $(X^4+1)$, tzn. dla $n=17$ i $W(X)=X^4+1$:

\begin{align}
    w1 &= 14x^3 + 10x^2 \\
    w2 &= 12x^2 + 10x + 6 \\
    w3 &= x^3 + x^2 + 11x + 7
\end{align}

Arytmetyka:

\begin{align}
    w1+w2 &= 14x^3 + 5x^2 + 10x + 6 \\
    w1*w2 &= 14x^3 + 9x^2 + 2x + 12 \\
    6*w3 &= 6x^3 + 6x^2 + 15x + 8
\end{align}

In [21]:
# your code here
import numpy as np

class ZnW:
    def __init__(self, N, W, p):
        self.N = N
        self.W = W
        self.r = self.calculate_polynomial(p)

    def calculate_polynomial(self, p):
        if not isinstance(p, np.ndarray):
            if isinstance(p, int):
                p = np.array([p])
            else:
                raise TypeError(f"Argument powinien być obiektem klasy numpy.ndarray lub liczbą całkowitą! Otrzymano typ: {type(p)}")

        _, remainder = np.polydiv(p, self.W)
        remainder_mod = []
        
        for coef in remainder:
            r = Zn(int(coef), self.N)
            remainder_mod.append(r.get_value())
            
        return np.array(remainder_mod)

    def check_argument(self, argument):
        if not isinstance(argument, ZnW):
            return ZnW(self.N, self.W, argument)
        elif argument.N != self.N:
            raise ValueError(f"Różne podstawy modularne N w argumentach: {self.N} oraz {argument.N}")
        elif not np.all(argument.W == self.W):
            raise ValueError(
                f"Różne wielomiany w pierścieniu ilorazowym: {self.polynomial_to_str(self.W)} oraz {self.polynomial_to_str(argument.W)}"
            )

        return argument

    def polynomial_to_str(self, poly):
        string = ""
        power = len(poly) - 1

        for i in range(len(poly)):
            coef = poly[i]
            
            if coef != 0:
                if i > 0: string += " + "
                if power - i > 1: string += f"{coef}x^{power - i}" if coef != 1 else f"x^{power - i}"
                elif power - i == 1: string += f"{coef}x" if coef != 1 else "x"
                else: string += f"{coef}"

        return string

    def __add__(self, other):
        other = self.check_argument(other)
        add_coeffs = np.polyadd(self.r, other.r)
        return ZnW(self.N, self.W, add_coeffs)

    def __radd__(self, other):
        return self + other

    def __mul__(self, other):
        other = self.check_argument(other)
        mul_coeffs = np.polymul(self.r, other.r)
        return ZnW(self.N, self.W, mul_coeffs)

    def __rmul__(self, other):
        return self * other

    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        return self.polynomial_to_str(self.r)

Przyjmuję konwencję: współczynniki podaję w tablicy NumPy'owej (`np.array`) od najstarszego (stojącego przy najwyższej potędze) do najmłodszego (wyrazu wolnego).

In [22]:
# TESTY
N = 17
W = np.array([1, 0, 0, 0, 1])

w1 = np.array([7, 0, 0, 14, 0, 0, 0])
w2 = np.array([24, 0, -5, -7, 13])
w3 = np.array([23, -3, 1, 35, 0, 4])

wn1 = ZnW(N, W, w1)
wn2 = ZnW(N, W, w2)
wn3 = ZnW(N, W, w3)

print(wn1)
print(wn2)
print(wn3)
print()

print(wn1 + wn2)
print(wn1 * wn2)
print(6 * wn3)

14x^3 + 10x^2
12x^2 + 10x + 6
x^3 + x^2 + 11x + 7

14x^3 + 5x^2 + 10x + 6
14x^3 + 9x^2 + 2x + 12
6x^3 + 6x^2 + 15x + 8
